## Set up the development environment

In [65]:
!pip install -q openai langchain beautifulsoup4 requests google-search-results kaggle huggingface-hub PyGithub

import os
import openai
import requests
from bs4 import BeautifulSoup
from serpapi import GoogleSearch
import time
import random
import pandas as pd
from kaggle.api.kaggle_api_extended import KaggleApi
from huggingface_hub import HfApi
from github import Github, RateLimitExceededException
from google.colab import userdata

In [66]:
# Upload kaggle.json file
from google.colab import files
print("Please upload your kaggle.json file.")
uploaded = files.upload()

# Set up the Kaggle API
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("Kaggle API key is configured.")

Please upload your kaggle.json file.


Saving kaggle.json to kaggle.json
Kaggle API key is configured.


In [67]:
# Set API keys from Colab Secrets
try:
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
except userdata.SecretNotFoundError:
    raise ValueError("OpenAI API Key not found in Colab Secrets. Please add it.")

try:
    os.environ["SERPAPI_API_KEY"] = userdata.get("SERPAPI_API_KEY")
except userdata.SecretNotFoundError:
    raise ValueError("SerpApi API Key not found in Colab Secrets. Please add it.")

try:
    os.environ["GITHUB_TOKEN"] = userdata.get("GITHUB_TOKEN")
except userdata.SecretNotFoundError:
    print("Warning: GitHub token not found in Colab Secrets. GitHub search will be skipped.")

try:
    os.environ["HF_TOKEN"] = userdata.get("HF_TOKEN")
except userdata.SecretNotFoundError:
    print("Warning: Hugging Face token not found in Colab Secrets. Hugging Face search will be skipped.")

print("API keys are set.")

API keys are set.


## Develop the research agent


In [68]:
def research_agent_run(company_or_industry, max_results=3):
    """
    Performs a real-time web search for the given company or industry using SerpApi,
    and scrapes content from the top search results with improved anti-bot handling.

    Args:
        company_or_industry (str): The name of the company or industry to research.
        max_results (int): Number of top search results to scrape.

    Returns:
        list of dict: Each dict contains 'url', 'title', and 'text' fields.
    """
    print(f"Running research agent for: {company_or_industry}")
    search_query = f"{company_or_industry} company profile and recent news"
    params = {
        "engine": "google",
        "q": search_query,
        "api_key": os.getenv("SERPAPI_API_KEY")
    }

    try:
        search = GoogleSearch(params)
        results = search.get_dict()

        # Check for SerpApi error
        if "error" in results:
            print(f"SerpApi Error: {results['error']}")
            return []

        organic_results = results.get("organic_results", [])
        if not organic_results:
            print("No organic results found.")
            return []

        scraped_docs = []

        # Commonly accepted headers to mimic a browser
        headers_template = {
            "User-Agent": random.choice([
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0 Safari/537.36",
                "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Safari/605.1.15",
                "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0 Safari/537.36"
            ]),
            "Accept-Language": "en-US,en;q=0.9",
            "Referer": "https://www.google.com/",
            "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
            "Connection": "keep-alive"
        }

        session = requests.Session()

        for i, result in enumerate(organic_results[:max_results]):
            url = result.get("link")
            title = result.get("title")
            if not url or not title:
                continue

            try:
                # Add a small random sleep to avoid bot detection
                time.sleep(random.uniform(2, 5))

                response = session.get(url, headers=headers_template, timeout=15)
                if response.status_code == 200:
                    soup = BeautifulSoup(response.text, "html.parser")
                    paragraphs = soup.find_all("p")
                    text = " ".join(p.get_text() for p in paragraphs)
                    clean_text = " ".join(text.split()).strip()

                    if clean_text:
                        scraped_docs.append({
                            "url": url,
                            "title": title,
                            "text": clean_text
                        })
                        print(f"Scraped: {url}")
                elif response.status_code == 403:
                    print(f"403 Forbidden at {url}. Skipping...")
                else:
                    print(f"Failed {url} — Status {response.status_code}")

            except requests.exceptions.RequestException as e:
                print(f"Error scraping {url}: {e}")

        return scraped_docs

    except Exception as e:
        print(f"Unexpected error in research agent: {e}")
        return []

# --- Quick test ---
if __name__ == "__main__":
    test_company = "google"
    research_docs = research_agent_run(test_company, max_results=3)
    if research_docs:
        print("\n Sample scraped documents:")
        for doc in research_docs:
            print(f"Title: {doc['title']}")
            print(f"URL: {doc['url']}")
            print(f"Text snippet: {doc['text'][:200]}...\n")
    else:
        print("No documents found.")


Running research agent for: google
Scraped: https://about.google/
Failed https://finance.yahoo.com/quote/GOOG/profile/ — Status 404
Scraped: https://www.forbes.com/companies/google/

 Sample scraped documents:
Title: About Google: Our products, technology and company ...
URL: https://about.google/
Text snippet: Jump to Content Explore our products and features across Search, Google Workspace and more Learn all about our leading AI models â and discover their capabilities See how weâre tackling some of th...

Title: Google | GOOG Stock Price, Company Overview & News
URL: https://www.forbes.com/companies/google/
Text snippet: Upgrade to gain access to exclusive features and add your profile photo. Learn More Forbes does not accept payment for placement on lists....



## Develop the market standards & use case generation agent


In [69]:
def usecase_agent_generate_usecases(company_name, research_findings):
    """
    Analyzes research findings and proposes relevant AI/GenAI use cases
    using a language model.

    Args:
        company_name (str): The name of the company.
        research_findings (list): A list of dictionaries containing research data.

    Returns:
        A list of dictionaries, each representing a proposed use case.
    """
    company_context = "\n".join([d.get('text', '') for d in research_findings if d.get('text')])

    if not company_context:
        print("Warning: No research context available to generate use cases.")
        return []

    prompt = f"""
You are an AI strategy consultant. Given these facts about {company_name} (context below), propose exactly 5 distinct GenAI/AI use cases for the company focusing on operations, customer experience, and monetization.

For each use case, provide the following information in a structured format:
TITLE: [Short Title]
DESCRIPTION: [One-sentence description]
DATA SOURCES: [List of required data sources, e.g., customer data, sales data, website logs]
BUSINESS IMPACT: [Expected business impact: Low, Medium, or High]
COMPLEXITY: [Estimated complexity: Low, Medium, or High]

Context:
{company_context}

Ensure each use case is clearly separated by a horizontal rule "---" and follows the exact 'FIELD_NAME: [Value]' format. Do not include any introductory or concluding text outside of the use case blocks.
"""

    try:
        client = openai.OpenAI()
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        llm_output = resp.choices[0].message.content

        use_cases = []
        use_case_blocks = llm_output.strip().split('---')

        for block in use_case_blocks:
            block = block.strip()
            if not block:
                continue

            current_use_case = {}
            lines = block.split('\n')
            for line in lines:
                line = line.strip()
                if line.startswith("TITLE:"):
                    current_use_case["title"] = line.replace("TITLE:", "").strip()
                elif line.startswith("DESCRIPTION:"):
                    current_use_case["description"] = line.replace("DESCRIPTION:", "").strip()
                elif line.startswith("DATA SOURCES:"):
                    current_use_case["data sources"] = line.replace("DATA SOURCES:", "").strip()
                elif line.startswith("BUSINESS IMPACT:"):
                    current_use_case["impact"] = line.replace("BUSINESS IMPACT:", "").strip()
                elif line.startswith("COMPLEXITY:"):
                    current_use_case["complexity"] = line.replace("COMPLEXITY:", "").strip()

            if current_use_case and current_use_case.get("title"):
                use_cases.append(current_use_case)

        if not use_cases or len(use_cases) < 3:
             print("Warning: Parsing failed or fewer than 3 valid use cases found.")
             print("LLM Output:\n", llm_output)

        return use_cases

    except Exception as e:
        print(f"Error generating use cases: {e}")
        return []

# Test with the documents from the previous cell
if 'research_docs' in locals() and research_docs:
    generated_use_cases = usecase_agent_generate_usecases(test_company, research_docs)
    print("\nGenerated Use Cases:")
    for uc in generated_use_cases:
        print(uc)


Generated Use Cases:
{'title': 'Intelligent Customer Support Chatbots', 'description': 'Deploy AI-driven chatbots to provide real-time support to users across Google Workspace and Search, enhancing user satisfaction and reducing response times.', 'data sources': 'Customer interaction logs, support ticket data, user feedback, conversation transcripts', 'impact': 'High', 'complexity': 'Medium'}
{'title': 'Predictive Analytics for Ad Spend Optimization', 'description': 'Utilize AI to analyze historical ad performance data and predict the most effective allocation of advertising budgets across platforms to maximize ROI.', 'data sources': 'Ad performance data, user engagement metrics, market trends, competitor analysis', 'impact': 'High', 'complexity': 'High'}
{'title': 'Personalized Content Recommendations', 'description': 'Implement generative AI algorithms to curate personalized content suggestions for users based on their search history and interactions across Google services.', 'data 

## Develop the resource asset collection agent

In [70]:
# --- Agent Functionality ---

from kaggle.api.kaggle_api_extended import KaggleApi

def search_kaggle(keyword, max_results=2):
    """
    Searches for datasets on Kaggle using the Kaggle API,
    retrieving basic info like URL, title, downloads, and dataset size.
    """
    print(f"Searching Kaggle for datasets related to: {keyword}")
    results = []
    try:
        api = KaggleApi()
        api.authenticate()

        search_results = api.dataset_list(search=keyword, sort_by="hottest")

        if not search_results:
            print(f"No datasets found on Kaggle for '{keyword}'")
            return []

        for ds in search_results[:max_results]:
            dataset_info = {
                "url": f"https://www.kaggle.com/{ds.ref}",
                "title": getattr(ds, "title", "Untitled Dataset"),
                "notes": f"Kaggle Dataset - Downloads: {getattr(ds, 'downloadCount', 'N/A')}, Views: {getattr(ds, 'viewCount', 'N/A')}"
            }

            try:
                # Fetch file info for dataset size
                files_in_dataset = api.dataset_list_files(ds.ref)
                if files_in_dataset and hasattr(files_in_dataset, "datasetFiles"):
                    total_size_bytes = sum(
                        getattr(f, "totalBytes", 0) for f in files_in_dataset.datasetFiles
                    )
                    total_size_mb = total_size_bytes / (1024 * 1024)
                    dataset_info["notes"] += f", Total Size: {total_size_mb:.2f} MB"
                else:
                    dataset_info["notes"] += ", Size info not available"

            except Exception as e:
                print(f"  - Error getting file list for {ds.ref}: {e}")
                dataset_info["notes"] += ", Error getting size info"

            results.append(dataset_info)

        return results

    except Exception as e:
        print(f"Error searching Kaggle: {e}")
        return []

def search_huggingface(keyword, max_results=2):
    """
    Searches for real datasets on Hugging Face using the Hugging Face Hub API.
    """
    print(f"Searching Hugging Face for datasets related to: {keyword}")
    results = []
    try:
        api = HfApi()
        datasets = api.list_datasets(search=keyword, sort="downloads", limit=max_results)

        for ds in datasets:
            results.append({
                "url": f"https://huggingface.co/datasets/{ds.id}",
                "title": ds.id.split('/')[-1],
                "notes": f"HuggingFace Dataset, downloads: {ds.downloads}"
            })
        return results
    except Exception as e:
        print(f"Error searching Hugging Face: {e}")
        return []

def search_github(keyword, max_results=2):
    """
    Searches for real GitHub repositories using the GitHub API.
    """
    print(f"Searching GitHub for repositories related to: {keyword}")
    github_token = os.getenv("GITHUB_TOKEN")
    if not github_token:
        print("GitHub token not found. Skipping GitHub search.")
        return []

    results = []
    try:
        from github import Github, RateLimitExceededException, Auth
        auth = Auth.Token(github_token)
        g = Github(auth=auth)

        query = f"{keyword} dataset" # Add " dataset" to the search query
        # Search for repositories, limiting results per page
        repositories = g.search_repositories(query=query, sort="stars", order="desc", per_page=max_results)


        for i, repo in enumerate(repositories):
            # The per_page parameter should handle limiting, but a safeguard loop is fine.
            if i >= max_results:
                break
            results.append({
                "url": repo.html_url,
                "title": repo.full_name,
                "notes": f"GitHub Repo, stars: {repo.stargazers_count}"
            })
        return results
    except RateLimitExceededException:
        print("GitHub API rate limit exceeded. Please wait or use a token with a higher limit.")
        return []
    except Exception as e:
        print(f"Error searching GitHub: {e}")
        return []


def dataset_agent_find_datasets(use_cases):
    """
    Searches for relevant datasets and resources for proposed use cases
    from Kaggle, Hugging Face, and GitHub.
    """
    updated_use_cases = []
    for uc in use_cases:
        keywords = []
        # Extract keywords from title and description, filter out short/common words
        all_words = []
        if uc.get("title"):
            all_words.extend(uc["title"].lower().replace(':', '').split())
        if uc.get("description"):
            all_words.extend(uc["description"].lower().replace(':', '').split())

        # Simple filtering of short words and basic stopwords
        stop_words = ['for', 'and', 'with', 'the', 'from', 'about', 'this', 'that', 'which', 'using', 'based', 'improve', 'enhance', 'generate', 'automate', 'predict', 'implement', 'utilize', 'leverage']
        keywords = [word for word in all_words if len(word) > 2 and word not in stop_words]


        search_keywords = " ".join(list(set(keywords))[:5]) # Take up to 5 unique keywords

        datasets = []
        # Call the corrected search functions
        datasets.extend(search_kaggle(search_keywords))
        datasets.extend(search_huggingface(search_keywords))
        datasets.extend(search_github(search_keywords))

        uc['datasets'] = datasets
        updated_use_cases.append(uc)
    return updated_use_cases

## Develop the prioritization and final proposal component

In [71]:
import pandas as pd

def prioritizer_rank(use_cases):
    """
    Ranks use cases based on impact and complexity.
    Uses a simplified scoring based on the provided formula structure.
    """
    ranked_use_cases = []
    for uc in use_cases:
        # Assign numerical scores
        impact_score = 0
        if 'impact' in uc:
            impact_lower = uc['impact'].lower()
            if 'high' in impact_lower:
                impact_score = 3
            elif 'med' in impact_lower:
                impact_score = 2
            elif 'low' in impact_lower:
                impact_score = 1

        complexity_score = 0
        if 'complexity' in uc:
            complexity_lower = uc['complexity'].lower()
            if 'high' in complexity_lower:
                complexity_score = 1 # Inverse score for complexity
            elif 'med' in complexity_lower:
                complexity_score = 2
            elif 'low' in complexity_lower:
                complexity_score = 3

        # Assuming data availability is medium (score 2) if not explicitly provided
        # A more sophisticated approach would analyze data sources for availability
        data_avail_score = 2

        # Calculate core score using the provided formula structure
        # core = 0.5 * impact_score + 0.4 * data_avail_score - 0.3 * complexity_score
        # Simplified formula using only impact and complexity as data_avail_score is assumed constant
        core_score = 0.5 * impact_score - 0.3 * complexity_score

        uc['core_score'] = core_score
        ranked_use_cases.append(uc)

    # Sort by core score in descending order
    ranked_use_cases = sorted(ranked_use_cases, key=lambda x: x.get('core_score', 0), reverse=True)

    return ranked_use_cases

def writer_save_markdown(use_cases, filename):
    """
    Formats and saves the prioritized use cases as a markdown report.
    """
    markdown_output = "# Prioritized AI/GenAI Use Case Proposal\n\n"

    for i, uc in enumerate(use_cases):
        markdown_output += f"## {i+1}. {uc.get('title', 'Untitled Use Case')}\n\n"
        markdown_output += f"**Description:** {uc.get('description', 'N/A')}\n\n"
        markdown_output += f"**Required Data Sources:** {uc.get('data sources', 'N/A')}\n\n"
        markdown_output += f"**Expected Business Impact:** {uc.get('impact', 'N/A')} | **Estimated Complexity:** {uc.get('complexity', 'N/A')}\n\n"

        if uc.get('datasets'):
            markdown_output += "**Relevant Resources:**\n"
            for dataset in uc['datasets']:
                title = dataset.get('title', 'Link')
                url = dataset.get('url', '#') # Use '#' as a fallback URL
                notes = dataset.get('notes', '')
                markdown_output += f"- [{title}]({url}) ({notes})\n"
            markdown_output += "\n" # Add an extra newline after resources

        markdown_output += "---\n\n" # Separator between use cases

    try:
        with open(filename, 'w') as f:
            f.write(markdown_output)
        print(f"Markdown report saved to {filename}")
    except IOError as e:
        print(f"Error saving markdown report to {filename}: {e}")

# Test the functions with the use_cases_with_datasets variable
if 'use_cases_with_datasets' in locals() and use_cases_with_datasets:
    prioritized_usecases = prioritizer_rank(use_cases_with_datasets)
    print("\nPrioritized Use Cases (with scores):")
    for uc in prioritized_usecases:
        print(f"Score: {uc.get('core_score', 'N/A'):.2f} - {uc.get('title', 'Untitled')}")

    writer_save_markdown(prioritized_usecases, 'prioritized_usecases.md')
else:
    print("Skipping prioritization and markdown writing: use_cases_with_datasets not found or empty.")



Prioritized Use Cases (with scores):
Score: 1.20 - Smart Ad Optimization
Score: 0.90 - Predictive Customer Support
Score: 0.90 - Personalized Content Recommendations
Score: 0.70 - AI-Driven Market Analysis
Score: 0.40 - Enhanced Workspace Collaboration Tools
Markdown report saved to prioritized_usecases.md


## Integrate the agents

In [72]:
import os

def orchestrator(company_or_industry):
    """
    Orchestrates the multi-agent system workflow.

    Args:
        company_or_industry: The name of the company or industry to research.

    Returns:
        A list of prioritized use cases with associated data and resources.
    """
    print(f"Starting orchestration for: {company_or_industry}")

    # 1. Research Phase
    print("Running research agent...")
    research_docs = research_agent_run(company_or_industry)
    if not research_docs:
        print("Research phase failed or returned no documents.")
        return []

    # 2. Use Case Generation Phase
    print("Running use case generation agent...")
    generated_use_cases = usecase_agent_generate_usecases(company_or_industry, research_docs)
    if not generated_use_cases:
        print("Use case generation phase failed or returned no use cases.")
        return []

    # 3. Resource Collection Phase
    print("Running dataset agent...")
    use_cases_with_datasets = dataset_agent_find_datasets(generated_use_cases)
    if not use_cases_with_datasets:
         print("Dataset agent failed or returned no datasets.")
         # Continue with use cases without datasets if the agent fails
         use_cases_with_datasets = generated_use_cases


    # 4. Prioritization Phase
    print("Running prioritization agent...")
    prioritized_usecases = prioritizer_rank(use_cases_with_datasets)
    if not prioritized_usecases:
         print("Prioritization agent failed.")
         # Continue with the list from the previous step if prioritization fails
         prioritized_usecases = use_cases_with_datasets


    # 5. Report Writing Phase
    print("Saving markdown report...")
    # Ensure the outputs directory exists
    output_dir = "outputs"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    output_filename = os.path.join(output_dir, f"{company_or_industry.replace(' ', '_').lower()}_usecases.md")
    writer_save_markdown(prioritized_usecases, output_filename)

    print(f"Orchestration complete. Report saved to {output_filename}")

    return prioritized_usecases

# Example of how to run the orchestrator
# orchestrated_results = orchestrator("Tech Innovations Inc.")
# print("\nFinal Prioritized Use Cases:")
# for uc in orchestrated_results:
#     print(f"{uc.get('core_score', 'N/A'):.2f} - {uc.get('title', 'Untitled')}")


In [73]:
# Test the orchestrator function
orchestrated_results = orchestrator("Global Solutions Corp")
print("\nFinal Prioritized Use Cases:")
for uc in orchestrated_results:
    print(f"Score: {uc.get('core_score', 'N/A'):.2f} - {uc.get('title', 'Untitled')}")


Starting orchestration for: Global Solutions Corp
Running research agent...
Running research agent for: Global Solutions Corp
Scraped: https://www.bloomberg.com/profile/company/0710124D:US
403 Forbidden at https://pitchbook.com/profiles/company/60178-42. Skipping...
403 Forbidden at https://www.zoominfo.com/c/global-solutions-inc/345259630. Skipping...
Running use case generation agent...
Running dataset agent...
Searching Kaggle for datasets related to: data financial market insights analyze
Searching Hugging Face for datasets related to: data financial market insights analyze
Searching GitHub for repositories related to: data financial market insights analyze
Searching Kaggle for datasets related to: preferences. clients personalized financial profiles
Searching Hugging Face for datasets related to: preferences. clients personalized financial profiles
Searching GitHub for repositories related to: preferences. clients personalized financial profiles
Searching Kaggle for datasets relat

## Build a user interface


In [74]:
import streamlit as st

# Ensure the orchestrator function is defined (it was defined in the previous step)
# if 'orchestrator' not in locals():
#     st.error("Orchestrator function not found. Please ensure the previous steps were run.")
# else:
st.title("Market Research & GenAI Use Case Agent")

company = st.text_input("Enter company name or industry:")

if st.button("Generate Use Cases"):
    if not company:
        st.warning("Please enter a company name or industry.")
    # Check if orchestrator function is available before calling
    elif 'orchestrator' in locals() and callable(orchestrator):
        st.info(f"Running analysis for {company}...")
        # Add a spinner or progress indicator for better UX
        with st.spinner('Generating use cases...'):
            prioritized_usecases = orchestrator(company)

        if prioritized_usecases:
            st.success("Analysis complete! Prioritized Use Cases:")
            for i, uc in enumerate(prioritized_usecases):
                st.subheader(f"{i+1}. {uc.get('title', 'Untitled Use Case')}")
                st.write(f"**Core Score:** {uc.get('core_score', 'N/A'):.2f}")
                st.write(f"**Description:** {uc.get('description', 'N/A')}")
                st.write(f"**Required Data Sources:** {uc.get('data sources', 'N/A')}")
                st.write(f"**Expected Business Impact:** {uc.get('impact', 'N/A')} | **Estimated Complexity:** {uc.get('complexity', 'N/A')}")

                if uc.get('datasets'):
                    st.write("**Relevant Resources:**")
                    for dataset in uc['datasets']:
                        title = dataset.get('title', 'Link')
                        url = dataset.get('url', '#')
                        notes = dataset.get('notes', '')
                        # Display as a clickable link
                        st.markdown(f"- [{title}]({url}) ({notes})")
                st.markdown("---") # Separator
        else:
            st.warning("Could not generate use cases for the provided company/industry.")
    else:
        st.error("Orchestrator function not found or not callable.")


2025-09-12 13:40:56.807 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 13:40:56.811 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 13:40:56.814 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 13:40:56.816 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 13:40:56.817 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 13:40:56.818 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 13:40:56.821 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-12 13:40:56.822 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [77]:
# Write the Streamlit app code to a file
streamlit_code = """
import streamlit as st
import os
import sys

# Ensure the orchestrator function and its dependencies are available
# Add the notebook directory to the path (for Colab/Notebook -> Streamlit integration)
notebook_dir = '/content'
if notebook_dir not in sys.path:
    sys.path.append(notebook_dir)

try:
    from __main__ import (
        orchestrator,
        research_agent_run,
        usecase_agent_generate_usecases,
        dataset_agent_find_datasets,
        prioritizer_rank,
        writer_save_markdown,
        search_kaggle,
        search_huggingface,
        search_github
    )
    print("Successfully imported notebook functions.")
except ImportError as e:
    st.error(f"Could not import notebook functions. Ensure notebook cells are run. Error: {e}")
    orchestrator = None

st.title("Market Research & GenAI Use Case Agent")

company = st.text_input("Enter company name or industry:")

if st.button("Generate Use Cases"):
    if not company:
        st.warning(" Please enter a company name or industry.")
    elif orchestrator is not None and callable(orchestrator):
        st.info(f"Running analysis for **{company}** ...")
        with st.spinner('Generating use cases...'):
            prioritized_usecases = orchestrator(company)

        if prioritized_usecases:
            st.success("Analysis complete! Prioritized Use Cases:")
            for i, uc in enumerate(prioritized_usecases):
                st.subheader(f"{i+1}. {uc.get('title', 'Untitled Use Case')}")
                st.write(f"**Core Score:** {uc.get('core_score', 'N/A')}")
                st.write(f"**Description:** {uc.get('description', 'N/A')}")
                st.write(f"**Required Data Sources:** {uc.get('data sources', 'N/A')}")
                st.write(f"**Expected Business Impact:** {uc.get('impact', 'N/A')} | **Estimated Complexity:** {uc.get('complexity', 'N/A')}")

                if uc.get('datasets'):
                    st.write("**Relevant Resources:**")
                    for dataset in uc['datasets']:
                        title = dataset.get('title', 'Link')
                        url = dataset.get('url', '#')
                        notes = dataset.get('notes', '')
                        st.markdown(f"- [{title}]({url}) ({notes})")

                st.markdown("---")  # Separator
        else:
            st.warning("Could not generate use cases. Check logs for errors.")
    else:
        st.error("Orchestrator function is not available. Please run the notebook cells first.")
"""

with open("app.py", "w") as f:
    f.write(streamlit_code)

print("Streamlit app code written to app.py")


Streamlit app code written to app.py


In [78]:
# Install streamlit_colab for easy launching in Colab
!pip install -q streamlit_colab

# Launch the Streamlit app
from streamlit_colab import run_streamlit
from google.colab import userdata # Import userdata to access secrets

# Get the ngrok token from Colab secrets
try:
    ngrok_token = userdata.get("NGROK_TOKEN")
except userdata.SecretNotFoundError:
    print("NGROK_TOKEN not found in Colab Secrets. Please add it to run Streamlit.")
    ngrok_token = None # Set token to None if not found

# Run Streamlit only if the ngrok token is available
if ngrok_token:
    run_streamlit("app.py", ngrok_token=ngrok_token)
else:
    print("Streamlit app cannot be launched without the NGROK_TOKEN.")

Public URL: https://199f84295a9e.ngrok-free.app


## Test and refine the system

In [76]:
# 1. Call the research_agent_run function with a sample company name
sample_company_name = "tesla"
sample_research_docs = research_agent_run(sample_company_name)

# 2. Print the sample_research_docs variable to inspect its content
print("\n--- Research Documents ---")
print(sample_research_docs)

# 3. Call the usecase_agent_generate_usecases function
sample_generated_use_cases = usecase_agent_generate_usecases(sample_company_name, sample_research_docs)

# 4. Print the sample_generated_use_cases variable to inspect its content
print("\n--- Generated Use Cases ---")
for uc in sample_generated_use_cases:
    print(uc)

Running research agent for: tesla
403 Forbidden at https://www.tesla.com/blog. Skipping...
Scraped: https://finance.yahoo.com/quote/TSLA/profile/
Scraped: https://www.forbes.com/companies/tesla/

--- Research Documents ---
[{'url': 'https://finance.yahoo.com/quote/TSLA/profile/', 'title': 'Tesla, Inc. (TSLA) Company Profile & Facts - Yahoo Finance', 'text': 'Oops, something went wrong Add holdings Tesla, Inc. designs, develops, manufactures, leases, and sells electric vehicles, and energy generation and storage systems in the United States, China, and internationally. The company operates in two segments, Automotive; and Energy Generation and Storage. The Automotive segment offers electric vehicles, as well as sells automotive regulatory credits; and non-warranty after-sales vehicle, used vehicles, body shop and parts, supercharging, retail merchandise, and vehicle insurance services. This segment also provides sedans and sport utility vehicles through direct and used vehicle sales, a 